In [2]:
import re
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,StratifiedKFold

In [4]:
df_sourceData = pd.read_csv('./广医-现病史.csv')
print(df_sourceData.info())
df_sourceData.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 36 columns):
现病史              2999 non-null object
经常咳嗽_稳定期         2999 non-null object
经常咳嗽时间           2999 non-null object
经常喘息_稳定期         2999 non-null object
经常喘息时间           2999 non-null object
经常胸闷_稳定期         2999 non-null object
经常胸闷时间           2999 non-null object
经常活动后气促_稳定期      2999 non-null object
经常活动后气促时间        2999 non-null object
经常咳痰_稳定期         2999 non-null object
经常咳痰时间           2999 non-null object
经常咳痰痰量           2999 non-null object
诱因_加重期           2999 non-null object
时间_加重期           2999 non-null object
咳嗽_加重期           2999 non-null object
咳嗽_时间            2999 non-null object
咳痰_加重期           2999 non-null object
咳痰_时间            2999 non-null object
胸闷_加重期           2999 non-null object
胸闷_时间            2999 non-null object
喘息_加重期           2999 non-null object
喘息_时间            2999 non-null object
气促_加重期           2999 non-null object
气促_时间            29

,现病史,经常咳嗽_稳定期,经常咳嗽时间,经常喘息_稳定期,经常喘息时间,经常胸闷_稳定期,经常胸闷时间,经常活动后气促_稳定期,经常活动后气促时间,经常咳痰_稳定期,...,发热_加重期,发热_时间,胸痛_加重期,胸痛_时间,心悸_加重期,心悸_时间,夜间阵发性呼吸困难_加重期,夜间阵发性呼吸困难_时间,双下肢水肿_加重期,双下肢水肿_时间
0,患者约10年余前无明显诱因出现活动后气促，伴少咳嗽，咳少量白色粘痰。无胸痛，无发热，无...,有,10年,未提及,未提及,无,无,有,10年,有,...,无,无,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及
1,患者10日前无明显诱因出现发热，体温最高至39.7度，伴咳嗽咳痰，痰为黄色粘痰，不易咳出，伴...,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及,...,有,10天,有,10天,无,无,未提及,未提及,未提及,未提及
2,患者于7余年前无明显反复出现咳嗽、咳痰、气促、心悸，伴夜间阵发性呼吸困难，曾在当地医院就医，...,有,7年,未提及,未提及,未提及,未提及,未提及,未提及,有,...,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及
3,患者于１年余前活动后出现气促，平素步行至3楼便觉气促，休息后可以缓解，无头晕、头痛，无咳嗽、...,无,无,未提及,未提及,未提及,未提及,有,1年,无,...,无,无,未提及,未提及,未提及,未提及,有,7天,未提及,未提及
4,患儿于8年前无明显诱因出现咳嗽，非犬吠样咳，咳嗽多于季节转变时加重。6年前出现痰多，为黄色粘...,有,8年,未提及,未提及,未提及,未提及,未提及,未提及,有,...,有,5天,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及


In [5]:
df_sourceData.columns

Index(['现病史', '经常咳嗽_稳定期', '经常咳嗽时间', '经常喘息_稳定期', '经常喘息时间', '经常胸闷_稳定期', '经常胸闷时间',
       '经常活动后气促_稳定期', '经常活动后气促时间', '经常咳痰_稳定期', '经常咳痰时间', '经常咳痰痰量', '诱因_加重期',
       '时间_加重期', '咳嗽_加重期', '咳嗽_时间', '咳痰_加重期', '咳痰_时间', '胸闷_加重期', '胸闷_时间',
       '喘息_加重期', '喘息_时间', '气促_加重期', '气促_时间', '咯血_加重期', '咯血_时间', '发热_加重期',
       '发热_时间', '胸痛_加重期', '胸痛_时间', '心悸_加重期', '心悸_时间', '夜间阵发性呼吸困难_加重期',
       '夜间阵发性呼吸困难_时间', '双下肢水肿_加重期', '双下肢水肿_时间'],
      dtype='object')

In [6]:
df_sourceData = df_sourceData[['现病史','经常咳嗽_稳定期', '经常喘息_稳定期', '经常胸闷_稳定期',
       '经常活动后气促_稳定期', '经常咳痰_稳定期', '咳嗽_加重期', '咳痰_加重期', '胸闷_加重期',
       '喘息_加重期', '气促_加重期', '咯血_加重期', '发热_加重期', '胸痛_加重期', '心悸_加重期', 
        '夜间阵发性呼吸困难_加重期', '双下肢水肿_加重期']]
print(df_sourceData['现病史'].tolist()[:5])
df_sourceData.loc[:,'现病史'] = df_sourceData['现病史'].str.replace('\n','')
df_sourceData.loc[:,'现病史'] = df_sourceData['现病史'].str.replace(' ','')
df_sourceData.head()
print(df_sourceData['现病史'].tolist()[:5])

['   患者约10年余前无明显诱因出现活动后气促，伴少咳嗽，咳少量白色粘痰。无胸痛，无发热，无咯血、盗汗、消瘦。间有夜间阵发性呼吸困难，无心悸、胸闷，无双下肢浮肿，无少尿等。5年前于广州军区陆军总医院门诊就诊，拟“慢性阻塞性肺气肿”予“激素”治疗（具体诊治欠详），症状可稍缓解，但气促常反复，未经正规诊治。2年前，患者上述症状再发，气促加重，运动耐量进行性下降，遂来我院门诊就诊，拟“AECOPD”收入我科继续治疗，予激素抗炎、沐舒坦化痰、及氨茶碱和可必特平喘解痉等处理后好转出院。出院后规范用舒利迭等吸入试剂治疗。今年5月份患者因气促加重入住广州市陆军总医院加强治疗，好转后出院。1月前患者诉气促再次加重，活动后明显，咳嗽咳痰较前无明显加重。患者近一个月以来，无发热为，无咯血，无胸闷，无午后低热，胃纳、精神一般，二便正常，体重无明显增减。\n', '患者10日前无明显诱因出现发热，体温最高至39.7度，伴咳嗽咳痰，痰为黄色粘痰，不易咳出，伴右侧胸肋部胸痛，咳嗽时加重，伴气促，活动后加重，伴口干口苦，无畏寒、寒战，无胸闷心悸，无头晕头痛，无恶心呕吐。遂于清远市中医院治疗，当地CT示：考虑右下肺叶炎症并脓肿形成。遂予以退热、抗感染（用过头孢哌酮舒巴坦7天，亚胺培南3天）、化痰、支持等治疗后，患者仍然反复发热，无明显好转。今患者为求进一步诊治入我院求诊，门诊以“右肺炎并脓肿”收住我科。入院后患者精神、食欲、睡眠一般，小便无异常，可见褐色大便。起病来患者体重无明显增减。', '患者于7余年前无明显反复出现咳嗽、咳痰、气促、心悸，伴夜间阵发性呼吸困难，曾在当地医院就医，诊断为“冠心病 陈旧性心肌梗死 ”，经治疗（具体不详）好转后出院。2月前上述症状加重，休息时既有发作，夜间不能平卧，无双下肢浮肿，即到当地医院住院治疗，予以对症治疗后症状好转，但出院后上诉症状反复发作，伴夜间阵发性呼吸困难及端坐呼吸，体力活动明显受限，伴咳嗽，咳痰，为白色稀痰，无血丝痰，无双下肢浮肿。无畏寒、发热，无恶心、呕吐，见有胸痛、心悸，无黑朦、晕厥、抽搐，为求进一步诊治入我院就医，门诊以“冠心病”收入我科治疗。患者起病以来，饮食稍差，睡眠一般，大小便正常，体重无明显改变。\n', '患者于１年余前活动后出现气促，平素步行至3楼便觉气促，休息后可以缓解，无头晕、头痛，无咳嗽、咳痰，无恶心、呕吐,间中端坐呼吸，

In [7]:
list_text = df_sourceData['现病史'].tolist()
list_text = [x.replace("\n","") for x in list_text]
print(list_text[:5])

['患者约10年余前无明显诱因出现活动后气促，伴少咳嗽，咳少量白色粘痰。无胸痛，无发热，无咯血、盗汗、消瘦。间有夜间阵发性呼吸困难，无心悸、胸闷，无双下肢浮肿，无少尿等。5年前于广州军区陆军总医院门诊就诊，拟“慢性阻塞性肺气肿”予“激素”治疗（具体诊治欠详），症状可稍缓解，但气促常反复，未经正规诊治。2年前，患者上述症状再发，气促加重，运动耐量进行性下降，遂来我院门诊就诊，拟“AECOPD”收入我科继续治疗，予激素抗炎、沐舒坦化痰、及氨茶碱和可必特平喘解痉等处理后好转出院。出院后规范用舒利迭等吸入试剂治疗。今年5月份患者因气促加重入住广州市陆军总医院加强治疗，好转后出院。1月前患者诉气促再次加重，活动后明显，咳嗽咳痰较前无明显加重。患者近一个月以来，无发热为，无咯血，无胸闷，无午后低热，胃纳、精神一般，二便正常，体重无明显增减。', '患者10日前无明显诱因出现发热，体温最高至39.7度，伴咳嗽咳痰，痰为黄色粘痰，不易咳出，伴右侧胸肋部胸痛，咳嗽时加重，伴气促，活动后加重，伴口干口苦，无畏寒、寒战，无胸闷心悸，无头晕头痛，无恶心呕吐。遂于清远市中医院治疗，当地CT示：考虑右下肺叶炎症并脓肿形成。遂予以退热、抗感染（用过头孢哌酮舒巴坦7天，亚胺培南3天）、化痰、支持等治疗后，患者仍然反复发热，无明显好转。今患者为求进一步诊治入我院求诊，门诊以“右肺炎并脓肿”收住我科。入院后患者精神、食欲、睡眠一般，小便无异常，可见褐色大便。起病来患者体重无明显增减。', '患者于7余年前无明显反复出现咳嗽、咳痰、气促、心悸，伴夜间阵发性呼吸困难，曾在当地医院就医，诊断为“冠心病陈旧性心肌梗死”，经治疗（具体不详）好转后出院。2月前上述症状加重，休息时既有发作，夜间不能平卧，无双下肢浮肿，即到当地医院住院治疗，予以对症治疗后症状好转，但出院后上诉症状反复发作，伴夜间阵发性呼吸困难及端坐呼吸，体力活动明显受限，伴咳嗽，咳痰，为白色稀痰，无血丝痰，无双下肢浮肿。无畏寒、发热，无恶心、呕吐，见有胸痛、心悸，无黑朦、晕厥、抽搐，为求进一步诊治入我院就医，门诊以“冠心病”收入我科治疗。患者起病以来，饮食稍差，睡眠一般，大小便正常，体重无明显改变。', '患者于１年余前活动后出现气促，平素步行至3楼便觉气促，休息后可以缓解，无头晕、头痛，无咳嗽、咳痰，无恶心、呕吐,间中端坐呼吸，夜间阵发性呼吸困难

In [8]:
df_label = df_sourceData[['经常咳嗽_稳定期', '经常喘息_稳定期', '经常胸闷_稳定期',
       '经常活动后气促_稳定期', '经常咳痰_稳定期', '咳嗽_加重期', '咳痰_加重期', '胸闷_加重期',
       '喘息_加重期', '气促_加重期', '咯血_加重期', '发热_加重期', '胸痛_加重期', '心悸_加重期', 
        '夜间阵发性呼吸困难_加重期', '双下肢水肿_加重期']]
df_label.head()

,经常咳嗽_稳定期,经常喘息_稳定期,经常胸闷_稳定期,经常活动后气促_稳定期,经常咳痰_稳定期,咳嗽_加重期,咳痰_加重期,胸闷_加重期,喘息_加重期,气促_加重期,咯血_加重期,发热_加重期,胸痛_加重期,心悸_加重期,夜间阵发性呼吸困难_加重期,双下肢水肿_加重期
0,有,未提及,无,有,有,未提及,未提及,无,未提及,未提及,无,无,未提及,未提及,未提及,未提及
1,未提及,未提及,未提及,未提及,未提及,有,有,无,未提及,有,未提及,有,有,无,未提及,未提及
2,有,未提及,未提及,未提及,有,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及,未提及
3,无,未提及,未提及,有,无,有,有,未提及,未提及,有,未提及,无,未提及,未提及,有,未提及
4,有,未提及,未提及,未提及,有,有,有,未提及,无,无,有,有,未提及,未提及,未提及,未提及


In [9]:
list_nodeName = list(df_label.columns)
print(list_nodeName)

['经常咳嗽_稳定期', '经常喘息_稳定期', '经常胸闷_稳定期', '经常活动后气促_稳定期', '经常咳痰_稳定期', '咳嗽_加重期', '咳痰_加重期', '胸闷_加重期', '喘息_加重期', '气促_加重期', '咯血_加重期', '发热_加重期', '胸痛_加重期', '心悸_加重期', '夜间阵发性呼吸困难_加重期', '双下肢水肿_加重期']


In [10]:
list_nodeValue = ['未提及','有','无']

In [12]:
for i in range(len(list_nodeValue)):
    df_label.replace(list_nodeValue[i], i, inplace=True)
    
print(df_label.head())

   经常咳嗽_稳定期  经常喘息_稳定期  经常胸闷_稳定期  经常活动后气促_稳定期  经常咳痰_稳定期  咳嗽_加重期  咳痰_加重期  \
0         1         0         2            1         1       0       0   
1         0         0         0            0         0       1       1   
2         1         0         0            0         1       0       0   
3         2         0         0            1         2       1       1   
4         1         0         0            0         1       1       1   

   胸闷_加重期  喘息_加重期  气促_加重期  咯血_加重期  发热_加重期  胸痛_加重期  心悸_加重期  夜间阵发性呼吸困难_加重期  \
0       2       0       0       2       2       0       0              0   
1       2       0       1       0       1       1       2              0   
2       0       0       0       0       0       0       0              0   
3       0       0       1       0       2       0       0              1   
4       0       2       2       1       1       0       0              0   

   双下肢水肿_加重期  
0          0  
1          0  
2          0  
3          0  
4          0  


/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3798: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


# 处理文本到预训练格式

In [59]:
list_splitSentencesOfText = []
for i in range(len(list_text)):
    sentences = re.split('(。|！|\!|？|\?)',list_text[i])
    new_sents = []
    for i in range(int(len(sentences)/2)):
        sent = sentences[2*i] + sentences[2*i+1]
        new_sents.append(sent)
    list_splitSentencesOfText.append(new_sents)
    
print(list_splitSentencesOfText[:2])

[['患者约10年余前无明显诱因出现活动后气促，伴少咳嗽，咳少量白色粘痰。', '无胸痛，无发热，无咯血、盗汗、消瘦。', '间有夜间阵发性呼吸困难，无心悸、胸闷，无双下肢浮肿，无少尿等。', '5年前于广州军区陆军总医院门诊就诊，拟“慢性阻塞性肺气肿”予“激素”治疗（具体诊治欠详），症状可稍缓解，但气促常反复，未经正规诊治。', '2年前，患者上述症状再发，气促加重，运动耐量进行性下降，遂来我院门诊就诊，拟“AECOPD”收入我科继续治疗，予激素抗炎、沐舒坦化痰、及氨茶碱和可必特平喘解痉等处理后好转出院。', '出院后规范用舒利迭等吸入试剂治疗。', '今年5月份患者因气促加重入住广州市陆军总医院加强治疗，好转后出院。', '1月前患者诉气促再次加重，活动后明显，咳嗽咳痰较前无明显加重。', '患者近一个月以来，无发热为，无咯血，无胸闷，无午后低热，胃纳、精神一般，二便正常，体重无明显增减。'], ['患者10日前无明显诱因出现发热，体温最高至39.7度，伴咳嗽咳痰，痰为黄色粘痰，不易咳出，伴右侧胸肋部胸痛，咳嗽时加重，伴气促，活动后加重，伴口干口苦，无畏寒、寒战，无胸闷心悸，无头晕头痛，无恶心呕吐。', '遂于清远市中医院治疗，当地CT示：考虑右下肺叶炎症并脓肿形成。', '遂予以退热、抗感染（用过头孢哌酮舒巴坦7天，亚胺培南3天）、化痰、支持等治疗后，患者仍然反复发热，无明显好转。', '今患者为求进一步诊治入我院求诊，门诊以“右肺炎并脓肿”收住我科。', '入院后患者精神、食欲、睡眠一般，小便无异常，可见褐色大便。', '起病来患者体重无明显增减。']]


In [60]:
f = open('./medical_text.txt','w')
for i in  range(len(list_splitSentencesOfText)):
    l_t = list_splitSentencesOfText[i]
    for ii in range(len(l_t)):    
        f.write(l_t[ii])
        f.write('\n')
    f.write('\n')
    
f.close()

In [61]:
df_sourceData_train,df_sourceData_val,df_sourceData_test = df_sourceData[:1500],df_sourceData[1500:2000],df_sourceData[2000:2999]
print(len(df_sourceData_test))
print(df_sourceData_test.head())

999
                                                    现病史 经常咳嗽_稳定期 经常喘息_稳定期  \
2000  患者5年前无明显诱因出现咳嗽，偶有黄色浓痰，无血丝痰，多在冬季及气温寒冷时发作，每次都伴有气...        有      未提及   
2001  患者1年前于在郑州某医院体检时，行CT检查示：右下肺阴影。一直未予治疗，门诊随访。7日前来我...      未提及      未提及   
2002  患者2年前在无诱因下出现咳嗽，起初症状轻，以干咳为主，无咳痰，无明显发热、畏寒，无胸闷、气促...        有      未提及   
2003  患者于20余年前无明显诱因下反复出现咳嗽，阵发性连声咳，咳少许白色泡沫痰，晨起明显，无咳血，...        有      未提及   
2004  患者入院前14年，常无明显诱因出现咳嗽、咳痰，阵发性，咳少许白色粘液性痰，无咳浓痰及血丝痰，...        有      未提及   

     经常胸闷_稳定期 经常活动后气促_稳定期 经常咳痰_稳定期 咳嗽_加重期 咳痰_加重期 胸闷_加重期 喘息_加重期 气促_加重期 咯血_加重期  \
2000        无         未提及        有      有    未提及    未提及    未提及      有    未提及   
2001      未提及         未提及      未提及      无      无      无    未提及      无    未提及   
2002        无         未提及        无      有      有    未提及    未提及      有      无   
2003      未提及           有        有      有      有    未提及    未提及      有      无   
2004        无           有        有    未提及    未提及    未提及    未提及    未提及    未提及   

     发热_加重期 胸痛_加重期 心悸_加重期 夜间阵发性呼吸困难_加重期 双下肢水肿_加重期  


In [62]:
df_sourceData_train.to_csv('medical_train.tsv',sep='\t',encoding='utf8')
df_sourceData_val.to_csv('medical_val.tsv',sep='\t',encoding='utf8')
df_sourceData_test.to_csv('medical_test.tsv',sep='\t',encoding='utf8')

# 16个现病史症状“有、无、未提及”分类BERT模型结果汇总

In [13]:
list_medicalModel_evalAccuracy = [0.97,0.954,0.954,0.942,0.95,0.918,0.876,0.93,0.972,0.898,0.93,0.942,0.922,
                                 0.942,0.968,0.946]


In [14]:
df_evalAccuracy = pd.DataFrame({"症状名称":list_nodeName,
                                "eval_accuracy":list_medicalModel_evalAccuracy
                               })

print(df_evalAccuracy)

             症状名称  eval_accuracy
0        经常咳嗽_稳定期          0.970
1        经常喘息_稳定期          0.954
2        经常胸闷_稳定期          0.954
3     经常活动后气促_稳定期          0.942
4        经常咳痰_稳定期          0.950
5          咳嗽_加重期          0.918
6          咳痰_加重期          0.876
7          胸闷_加重期          0.930
8          喘息_加重期          0.972
9          气促_加重期          0.898
10         咯血_加重期          0.930
11         发热_加重期          0.942
12         胸痛_加重期          0.922
13         心悸_加重期          0.942
14  夜间阵发性呼吸困难_加重期          0.968
15      双下肢水肿_加重期          0.946


In [15]:
# 平均准确率
print(sum(list_medicalModel_evalAccuracy)/16)

0.9383750000000001


# 构建深圳现病史数据测试

In [18]:
df_sourceData02 = pd.read_excel('./深圳-现病_处理后史.xlsx')
print(df_sourceData02.info())
df_sourceData02.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 48 columns):
现病史              3000 non-null object
经常咳嗽_稳定期         3000 non-null int64
经常咳嗽时间           3000 non-null object
经常喘息_稳定期         3000 non-null int64
经常喘息时间           3000 non-null object
经常胸闷_稳定期         3000 non-null int64
经常胸闷时间           3000 non-null object
经常活动后气促_稳定期      3000 non-null int64
经常活动后气促时间        3000 non-null object
经常咳痰_稳定期         3000 non-null int64
经常咳痰时间           3000 non-null object
经常咳痰痰量           3000 non-null object
诱因_加重期           3000 non-null object
时间_加重期           3000 non-null object
咳嗽_加重期           3000 non-null int64
咳嗽_时间            3000 non-null object
咳痰_加重期           3000 non-null int64
咳痰_时间            3000 non-null object
胸闷_加重期           3000 non-null int64
胸闷_时间            3000 non-null object
喘息_加重期           3000 non-null int64
喘息_时间            3000 non-null object
气促_加重期           3000 non-null int64
气促_时间            3000 non-nul

,现病史,经常咳嗽_稳定期,经常咳嗽时间,经常喘息_稳定期,经常喘息时间,经常胸闷_稳定期,经常胸闷时间,经常活动后气促_稳定期,经常活动后气促时间,经常咳痰_稳定期,...,胸闷,喘息,气促,咯血,发热,胸痛,心悸,夜间阵发性呼吸困难,双下肢水肿,首发症状n年前
0,患者1天前摔致右侧胸痛、气促，为持续性钝痛，偶有阵发性咳嗽，痰较多，无明显呼吸困难，无头痛、...,0,0,0,0,0,0,0,0,0,...,0,0,1,-1,0,1,-1,0,0,1天
1,患者7年前感冒后出现咳嗽，咳少量白色黏液痰，伴发热，体温最高38.5℃，伴畏寒、盗汗，伴鼻塞...,1,7年,-1,-1,-1,-1,0,0,1,...,-1,-1,-1,-1,1,-1,0,-1,0,7年
2,患者患者10年前开始，每于天气转凉或气候变化易出现咳嗽，咳少量白痰，无气促，无呼吸困难，每年...,1,10年,-1,-1,0,0,-1,-1,1,...,-1,-1,-1,-1,1,-1,-1,0,0,10年
3,患者缘于10余年前开始反复出现咳嗽，咳白粘痰，伴活动后气促，常可自行闻及喘鸣音，受凉感冒、天...,1,10年,1,10年,0,0,1,10年,1,...,-1,1,1,-1,-1,-1,0,0,0,10年
4,患者2年前开始反复受凉后咳嗽、咳痰、气喘，曾在我科住院，诊断“慢性阻塞性肺疾病急性加重期”，...,1,2年,1,2年,0,0,0,0,1,...,0,1,1,0,1,0,-1,0,0,2年


In [20]:
df_sourceData02 = df_sourceData02[['现病史','经常咳嗽_稳定期', '经常喘息_稳定期', '经常胸闷_稳定期',
       '经常活动后气促_稳定期', '经常咳痰_稳定期', '咳嗽_加重期', '咳痰_加重期', '胸闷_加重期',
       '喘息_加重期', '气促_加重期', '咯血_加重期', '发热_加重期', '胸痛_加重期', '心悸_加重期', 
        '夜间阵发性呼吸困难_加重期', '双下肢水肿_加重期']]
print(df_sourceData02.head())
print(df_sourceData02['现病史'].tolist()[:5])
df_sourceData02.loc[:,'现病史'] = df_sourceData02['现病史'].str.replace('\n','')
df_sourceData02.loc[:,'现病史'] = df_sourceData02['现病史'].str.replace(' ','')
print(df_sourceData02['现病史'].tolist()[:5])

                                                 现病史  经常咳嗽_稳定期  经常喘息_稳定期  \
0  患者1天前摔致右侧胸痛、气促，为持续性钝痛，偶有阵发性咳嗽，痰较多，无明显呼吸困难，无头痛、...         0         0   
1  患者7年前感冒后出现咳嗽，咳少量白色黏液痰，伴发热，体温最高38.5℃，伴畏寒、盗汗，伴鼻塞...         1        -1   
2  患者患者10年前开始，每于天气转凉或气候变化易出现咳嗽，咳少量白痰，无气促，无呼吸困难，每年...         1        -1   
3  患者缘于10余年前开始反复出现咳嗽，咳白粘痰，伴活动后气促，常可自行闻及喘鸣音，受凉感冒、天...         1         1   
4  患者2年前开始反复受凉后咳嗽、咳痰、气喘，曾在我科住院，诊断“慢性阻塞性肺疾病急性加重期”，...         1         1   

   经常胸闷_稳定期  经常活动后气促_稳定期  经常咳痰_稳定期  咳嗽_加重期  咳痰_加重期  胸闷_加重期  喘息_加重期  气促_加重期  \
0         0            0         0       1       0       0       0       1   
1        -1            0         1       0       0       0       0       0   
2         0           -1         1       1       1      -1      -1      -1   
3         0            1         1       1       1      -1       1       1   
4         0            0         1      -1      -1       0       1       1   

   咯血_加重期  发热_加重期  胸痛_加重期  心悸_加重期  夜间阵发性呼吸困难_加重期  双下肢水肿_加重期  
0      -1   

In [21]:
list_text02 = df_sourceData02['现病史'].tolist()
list_text02 = [x.replace("\n","") for x in list_text02]
print(list_text02[:5])

['患者1天前摔致右侧胸痛、气促，为持续性钝痛，偶有阵发性咳嗽，痰较多，无明显呼吸困难，无头痛、双下肢痛，无昏迷，无呕吐，无心慌、口渴，无咯血，无腹痛、腹泻、呕血、血便，当地医院X光片检查示“1.老年性心肺改变；慢支并感染2.右侧第8肋骨折，第7、9肋可疑骨折”，为求进一步诊治来我院就诊，门诊拟“肋骨骨折”收入我科。起病以来，食欲、精神尚可，体重无下降。二便正常。', '患者7年前感冒后出现咳嗽，咳少量白色黏液痰，伴发热，体温最高38.5℃，伴畏寒、盗汗，伴鼻塞、流涕，无气喘，无胸痛、胸闷，无夜间阵发性呼吸困难，至外院就诊，诊断“肺炎”，予抗感染、止咳、祛痰治疗，后症状缓解出院。患者7年间咳嗽、咳痰反复发作，曾多次于外院就诊，均予抗感染、止咳、祛痰治疗，症状均可缓解。2月前无明显诱因咳嗽再发，伴咳少量白色黏液痰，无咯血，无发热，无胸闷、胸痛，无夜间阵发性呼吸困难，现为进一步诊治至我院就诊，拟“咳嗽查因”收入我科。患者起病以来，饮食可，睡眠欠佳，大小便正常，体重有无明显改变。', '患者患者10年前开始，每于天气转凉或气候变化易出现咳嗽，咳少量白痰，无气促，无呼吸困难，每年发病大于3个月，曾多次于医院门诊或住院治疗，诊断为“慢性阻塞性肺疾病”，予抗感染、止咳化痰、解痉平喘等治疗后好转。长期吸入“信必可”，并长期家庭氧疗，平素上楼、上坡无气喘。9天前患者受凉后出现咳嗽、咳痰、发热，最高体温38.5℃，无畏寒、寒战，咳黄色脓痰，量较多，于附近社康输液治疗4天，未再出现发热，但咳嗽、咳痰仍未见明显改善，1月5日就诊我科门诊，给予“雾化、阿奇霉素静滴及莫西沙星口服”等对症处理，症状改善不明显，无胸闷、气喘，食欲欠佳，为求进一步诊治，入住我科，病程中，无咯血，无胸痛，无心慌，大小便正常，体重近期无下降。', '患者缘于10余年前开始反复出现咳嗽，咳白粘痰，伴活动后气促，常可自行闻及喘鸣音，受凉感冒、天气变化后易发，曾在我院诊断"慢性支气管炎、肺气肿"，予抗感染、平喘、止咳化痰等治疗后症状改善，但反复，曾短暂吸入舒利迭治疗。3月前再次出现咳嗽、咳白色泡沫痰，伴气促，我科住院治疗，诊断为"慢性阻塞性肺疾病急性加重期、肺部感染"，予莫西沙星抗感染、化痰、平喘等治疗好转后出院，出院后未规律服药。5天前咳嗽、咳痰较前加重，咳白色泡沫样黏痰，伴气喘，平卧后气喘加重，我院门诊予"阿奇霉素、克

In [22]:
df_label02 = df_sourceData02[['经常咳嗽_稳定期', '经常喘息_稳定期', '经常胸闷_稳定期',
       '经常活动后气促_稳定期', '经常咳痰_稳定期', '咳嗽_加重期', '咳痰_加重期', '胸闷_加重期',
       '喘息_加重期', '气促_加重期', '咯血_加重期', '发热_加重期', '胸痛_加重期', '心悸_加重期', 
        '夜间阵发性呼吸困难_加重期', '双下肢水肿_加重期']]
df_label02.head()

,经常咳嗽_稳定期,经常喘息_稳定期,经常胸闷_稳定期,经常活动后气促_稳定期,经常咳痰_稳定期,咳嗽_加重期,咳痰_加重期,胸闷_加重期,喘息_加重期,气促_加重期,咯血_加重期,发热_加重期,胸痛_加重期,心悸_加重期,夜间阵发性呼吸困难_加重期,双下肢水肿_加重期
0,0,0,0,0,0,1,0,0,0,1,-1,0,1,-1,0,0
1,1,-1,-1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,1,-1,0,-1,1,1,1,-1,-1,-1,-1,-1,-1,-1,0,0
3,1,1,0,1,1,1,1,-1,1,1,-1,-1,-1,0,0,0
4,1,1,0,0,1,-1,-1,0,1,1,0,-1,0,-1,0,0


In [23]:
df_sourceData02.replace(-1,'无',inplace=True)
df_sourceData02.replace(0,'未提及',inplace=True)
df_sourceData02.replace(1,'有',inplace=True)
print(df_sourceData02.head())

                                                 现病史 经常咳嗽_稳定期 经常喘息_稳定期  \
0  患者1天前摔致右侧胸痛、气促，为持续性钝痛，偶有阵发性咳嗽，痰较多，无明显呼吸困难，无头痛、...      未提及      未提及   
1  患者7年前感冒后出现咳嗽，咳少量白色黏液痰，伴发热，体温最高38.5℃，伴畏寒、盗汗，伴鼻塞...        有        无   
2  患者患者10年前开始，每于天气转凉或气候变化易出现咳嗽，咳少量白痰，无气促，无呼吸困难，每年...        有        无   
3  患者缘于10余年前开始反复出现咳嗽，咳白粘痰，伴活动后气促，常可自行闻及喘鸣音，受凉感冒、天...        有        有   
4  患者2年前开始反复受凉后咳嗽、咳痰、气喘，曾在我科住院，诊断“慢性阻塞性肺疾病急性加重期”，...        有        有   

  经常胸闷_稳定期 经常活动后气促_稳定期 经常咳痰_稳定期 咳嗽_加重期 咳痰_加重期 胸闷_加重期 喘息_加重期 气促_加重期 咯血_加重期  \
0      未提及         未提及      未提及      有    未提及    未提及    未提及      有      无   
1        无         未提及        有    未提及    未提及    未提及    未提及    未提及    未提及   
2      未提及           无        有      有      有      无      无      无      无   
3      未提及           有        有      有      有      无      有      有      无   
4      未提及         未提及        有      无      无    未提及      有      有    未提及   

  发热_加重期 胸痛_加重期 心悸_加重期 夜间阵发性呼吸困难_加重期 双下肢水肿_加重期  
0    未提及      有      无           未提及       

In [24]:
df_sourceData02.to_csv('medical_test02.tsv',sep='\t',encoding='utf8')

# 在深圳3000现病史数据上验证模型准确率

In [25]:
list_medicalModel_evalAccuracy02 = [0.9713,0.8956,0.956,0.8813,0.951,0.9056,0.859,0.9343,0.8773,0.8716,0.9023,0.9113,0.9286,
                                 0.9586,0.9776,0.9526]

In [26]:
df_evalAccuracy02 = pd.DataFrame({"症状名称":list_nodeName,
                                "eval_accuracy":list_medicalModel_evalAccuracy02
                               })

print(df_evalAccuracy02)

             症状名称  eval_accuracy
0        经常咳嗽_稳定期         0.9713
1        经常喘息_稳定期         0.8956
2        经常胸闷_稳定期         0.9560
3     经常活动后气促_稳定期         0.8813
4        经常咳痰_稳定期         0.9510
5          咳嗽_加重期         0.9056
6          咳痰_加重期         0.8590
7          胸闷_加重期         0.9343
8          喘息_加重期         0.8773
9          气促_加重期         0.8716
10         咯血_加重期         0.9023
11         发热_加重期         0.9113
12         胸痛_加重期         0.9286
13         心悸_加重期         0.9586
14  夜间阵发性呼吸困难_加重期         0.9776
15      双下肢水肿_加重期         0.9526


In [27]:
# 平均准确率
print(sum(list_medicalModel_evalAccuracy02)/16)

0.9208750000000002
